Copyright (c) 2023 [윤기태]

https://github.com/yoonkt200/python-data-analysis

[MIT License](https://github.com/yoonkt200/python-data-analysis/blob/master/LICENSE.txt)

# (가제) 파이썬 데이터 분석

-----

# <이것이 데이터 분석이다 with 파이썬> 4쇄 이후 변경된 크롤링 파트
- 망고플레이트 웹페이지 리뷰 데이터 크롤링

### 바로가기

- [<Step1. 크롤링> : 크롤링으로 웹 데이터 가져오기](#<Step1.-크롤링>-:-크롤링으로-웹-데이터-가져오기)
    - [망고플레이트 웹페이지]
    - [리뷰 정보 크롤링]

-----

In [1]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# <Step1. 크롤링> : 크롤링으로 웹 데이터 가져오기

### [망고플레이트 웹페이지]
- https://www.mangoplate.com/
- 맛집 관련 리뷰를 볼 수 있는 웹페이지
- 본 예제에서는 웹 페이지에서 "강남역" 이라는 검색어로 검색한 결과를 크롤링 합니다.

![앱 생성 페이지](img/3.png)

-----

- 검색 결과로 나오는 페이지들의 리스트를 크롤링합니다.

In [57]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

# brew 로 설치된 chromedriver의 path (Mac)
# mac_path = "/usr/local/bin/chromedriver"  # m1 이전 맥 OS
mac_path = "/opt/homebrew/bin/chromedriver"  # m1 이후 맥 OS

# 윈도우용 크롬 웹드라이버 실행 경로 (Windows)
window_path = "chromedriver.exe"

# 크롤링할 사이트 주소를 정의합니다. (사이트 검색창에 "강남역"을 입력한 url)
source_url = "https://www.mangoplate.com/search/%EA%B0%95%EB%82%A8%EC%97%AD"

# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
driver = webdriver.Chrome(mac_path)  # for Mac
# driver = webdriver.Chrome(window_path)  # for Windows
driver.get(source_url)
req = driver.page_source
soup = BeautifulSoup(req, "html.parser")
info_list = soup.find_all(name="div", attrs={"class":"info"})

In [58]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_urls = []
page_url_base = "https://www.mangoplate.com"
for index in range(0, len(info_list)):
    info = info_list[index]
    review_url = info.find(name="a")
    if review_url is not None:
        page_urls.append(page_url_base + review_url.get("href"))

# 중복 url을 제거합니다.
page_urls = list(set(page_urls))

# 다섯 개의 페이지를 출력합니다.
for page in page_urls[:5]:
    print(page)

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

https://www.mangoplate.com/restaurants/kZUywyY8DPyx
https://www.mangoplate.com/restaurants/udMS7prvVnNn
https://www.mangoplate.com/restaurants/0Ftpl9Mr_Isl
https://www.mangoplate.com/restaurants/0Vq0bf1rZJLZ
https://www.mangoplate.com/restaurants/9fX9iZIgEG7H


-----

### [리뷰 정보 크롤링]

![앱 생성 페이지](img/4.png)

-----

In [59]:
columns = ['score', 'review']
df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(mac_path)  # for Mac
# driver = webdriver.Chrome(window_path)  # for Windows
for page_url in page_urls:
    
    # 상세보기 페이지에 접속합니다
    driver.get(page_url)
    
    # 리뷰를 크롤링합니다
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_list = soup.find(name="section", attrs={"class":"RestaurantReviewList"})
    
    # 평가 정보('맛있다' 마크)를 가져옵니다.
    review_recommend_list = review_list.find_all(name="div", attrs={"class":"RestaurantReviewItem__Rating RestaurantReviewItem__Rating--Recommend"})
    
    # 리뷰를 가져옵니다.
    review_text_list = review_list.find_all(name="div", attrs={"class":"RestaurantReviewItem__ReviewContent"})
    
    for review_recommend, review_text in zip(review_recommend_list, review_text_list):
        row = [review_recommend.find(name="span").text, review_text.find(name="p").text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
        
driver.close()

In [60]:
df.head()

,score,review
0,맛있다,\n맛있게 먹었어요 !!그런데 신메뉴였던 버섯이랑 계란 들어간 샌드위치는들고먹어도 ...
1,맛있다,\n 한줄평: 정통 아메리칸 델리카트슨 다이너. 예랑Pick은 '파...
2,맛있다,\n 늘 웨이팅이 있어 궁금했던 카츠8 - 돈카츠가 주된 메뉴지만...
3,맛있다,\n .\n
4,맛있다,"\n 카츠반상 22000원치즈돈까스와 우동, 후토마끼가 같이 나와요..."


In [61]:
# '맛있다' 라는 마크가 달린 리뷰는 긍정 리뷰, 그 외는 부정 리뷰로 평가합니다.
df['y'] = df['score'].apply(lambda x: 1 if x == '맛있다' else 0)
print(df.shape)
df.head()

(5, 3)


,score,review,y
0,맛있다,\n맛있게 먹었어요 !!그런데 신메뉴였던 버섯이랑 계란 들어간 샌드위치는들고먹어도 ...,1
1,맛있다,\n 한줄평: 정통 아메리칸 델리카트슨 다이너. 예랑Pick은 '파...,1
2,맛있다,\n 늘 웨이팅이 있어 궁금했던 카츠8 - 돈카츠가 주된 메뉴지만...,1
3,맛있다,\n .\n,1
4,맛있다,"\n 카츠반상 22000원치즈돈까스와 우동, 후토마끼가 같이 나와요...",1


-----

## 크롤링 이후의 분석 내용은 책에 기재되어 있는 내용인 `02-restaurant-sentiment-analysis.ipynb` 파일과 동일합니다.